In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
import json
import math


In [3]:


# FIRST I WANT TO ITERATE THROUGH EVERY CATEGORY 
# THEN I WANT TO GET THE NUMBER OF PAGES IN EACH CATEGORY
# PERHAPS I SHOULD START BY GETTING ALL THE URLS APPENED AT ONCE, BECAUSE 
# CREATE A SEPARATE LIST CONTAINING LINKS OF ALL CATEGORIES - PAGE 1

#An example of the link i want to manage below : -
#https://www.e.leclerc/api/rest/live-api/product-search?language=fr-FR&size=90&sorts=%5B%5D&page=1&categories=%7B%22code%22:%5B%22NAVIGATION_bon-plan-sport-et-loisir%22%5D%7D


In [4]:

class LsportsSpider(scrapy.Spider):
    name = 'lsports'
    start_urls = ["https://www.e.leclerc/api/rest/live-api/categories-tree-by-code/NAVIGATION_sport-loisirs?pageType=NAVIGATION&maxDepth=undefined"]

    new_start = []

    headers = {
        "accept" : "application/json",
        "accept-encoding" : "gzip, deflate, br",
        "accept-language" : "en-US,en;q=0.9",
        "referer" : "https://www.e.leclerc/cat/sport-loisirs",
        "sec-fetch-dest" : "empty",
        "sec-fetch-mode" : "cors",
        "sec-fetch-site" : "same-origin",
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
        "X-Requested-With" : "Fetch",
    }
    catname = []
    cnam = ""
    
    def parse(self, response):
        url = "https://www.e.leclerc/api/rest/live-api/categories-tree-by-code/NAVIGATION_sport-loisirs?pageType=NAVIGATION&maxDepth=undefined"

        yield scrapy.Request(url, callback=self.parse_page, headers=self.headers)
    
  
    def parse_page(self, response):

        new_url1 = "https://www.e.leclerc/api/rest/live-api/product-search?language=fr-FR&size=30&sorts=%5B%5D&page="
        new_url2 = "&categories=%7B%22code%22:%5B%22"
        new_url3 = "%22%5D%7D"
        

        raw_data = response.body
        data = json.loads(raw_data)
        x = len(data["children"]) 

        for i in range(0,1):

            self.cnam = data["children"][i]['code']
            pagenum = math.ceil((data["children"][i]['nbProducts'] / 30)) + 1

            for z in range(1,pagenum):

                total_url = new_url1+str(z)+new_url2+str(data["children"][i]['code'])+new_url3
                yield scrapy.Request(total_url, callback=self.parse_page_new, headers=self.headers)
    


    def parse_page_new(self, response):

        new_raw = response.body
        data1 = json.loads(new_raw)

        y = len(data1["items"])
        for varx in range(y):
            yield {
                'name' : data1["items"][varx]['label']
                }



In [5]:
%%bash
scrapy runspider spider.py -o test1.csv

2023-07-26 17:21:47 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2023-07-26 17:21:47 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.6 (main, Nov 14 2022, 16:10:14) [GCC 11.3.0], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
2023-07-26 17:21:47 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2023-07-26 17:21:47 [py.warnings] WARNING: /home/shane/.local/lib/python3.10/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future ve

The spider runs and produces an output to **test1.csv**